You have time until the competition ends. Send it in before final.

The data given is a subset of how Kobe Bryant shoot for his entire NBA career.
I took out some data for the competition.

https://www.kaggle.com/c/kobe-bryant-shot-selection

Your report notebook should include
- some nice visualizations of the data.
- description of how each visualization affect the decision you made in making classifier
- The username you use in kaggle so I can identify your rank.

Grading will be in project category. It consists of two parts.

90% report
10% for ranking in Kaggle.
- 1% for entering the competion
- 3% if you beat 50% benchmark.
- 5% if you are top 50%
- 7% if you are top 20%
- 10% if you are in the leader board
- Cash prize (from someone else) if you win

You may study the notebook by other. You may use sk-learn etc.

In [72]:
%matplotlib inline
import os
import time
import sys
import numpy as np
from multiprocessing import Pool

In [73]:
def readFile(file):
    tmp = []
    with open(file) as f:
        for line in f:
            tmp.append(line)
    return tmp

def cleanData(data):
    return [d.rstrip('\n').split(',') for d in data[1:]]

def toFloat(data):
    for d in data:
        for i in xrange(len(d)):
            try:
                d[i] = float(d[i])
            except:
                pass
    return data
            
def test_train(data):
    test = []
    train = []
    np.random.seed(99999)
    rd = [np.random.randint(10) for x in xrange(len(data))]
    for i in xrange(len(data)):
        if rd[i] <=1:
            test.append(data[i])
        else:
            train.append(data[i])
    return test,train

In [102]:
data = readFile('data.csv')
data = cleanData(data)
fdata = toFloat(data)
test, train = test_train(fdata)

action_type,
combined_shot_type,
game_event_id,
game_id,
lat,

loc_x,
loc_y,
lon,
minutes_remaining,
period,

playoffs,
season,
seconds_remaining,
shot_distance,
shot_made_flag,

shot_type,
shot_zone_area,
shot_zone_basic,
shot_zone_range,
team_id,

team_name,
game_date,
matchup,
opponent,
shot_id

In [62]:
def pickData(data):
#     useIdx = [0, 1, 5, 6, 7, 13, 15, 16, 17, 18]

    useIdx = [5, 6, 7, 13]
    cIdx = 14
    new_data = []
    classes = []
    
    for d in data:
        c = d[cIdx]
        if c != '':
            new_data.append( [d[idx] for idx in useIdx] )
            classes.append(c)
    return np.array(new_data), np.array(classes)

In [63]:
new_train, classes = pickData(train)
new_test, test_classes = pickData(test)

In [64]:
print new_train[:10]
print new_test[:10]

[[-157.        0.     -118.4268   15.    ]
 [ 138.      175.     -118.1318   22.    ]
 [-145.      -11.     -118.4148   14.    ]
 [   0.        0.     -118.2698    0.    ]
 [ -65.      108.     -118.3348   12.    ]
 [ -94.      238.     -118.3638   25.    ]
 [ -67.      110.     -118.3368   12.    ]
 [ -23.       47.     -118.2928    5.    ]
 [  62.      192.     -118.2078   20.    ]
 [-132.       97.     -118.4018   16.    ]]
[[-101.      135.     -118.3708   16.    ]
 [   0.        0.     -118.2698    0.    ]
 [ -33.      125.     -118.3028   12.    ]
 [ 121.      127.     -118.1488   17.    ]
 [ -94.        4.     -118.3638    9.    ]
 [-117.      226.     -118.3868   25.    ]
 [   3.      144.     -118.2668   14.    ]
 [ -16.      110.     -118.2858   11.    ]
 [-109.      150.     -118.3788   18.    ]
 [ -58.      196.     -118.3278   20.    ]]


In [65]:
class Node:
    def __init__(self, msg='label'):
        self.left = None
        self.right = None
        self.msg = msg
        
    def is_leaf(self):
        return self.left is None and self.right is None
    
    def __str__(self):
        return self.msg
    
class DecisionNode(Node):
    def __init__(self, vid, cut, prob = -1):
        Node.__init__(self)
        self.vid = vid
        self.cut = cut
        self.prob = prob

    def __str__(self):
        if not self.is_leaf():
            return 'x%d < %.2f'%(self.vid, self.cut)
        else:
            return '%.2f'%self.prob
            
    def classify(self, data):
        if self.is_leaf():
            return self.prob
        else:
            if (data[self.vid] < self.cut):
                return self.left.classify(data)
            else:
                return self.right.classify(data)

def make_tree(data):
    features, cls, minleaf = data
    if len(features) < minleaf:
        return DecisionNode(None, None, sum(cls>0)/float(len(features)))
    elif sum(cls>0) == len(cls):
        return DecisionNode(None, None, 1 if cls[0] > 0 else 0)
    else:
        vid, cut, s, left, ftLeft, right, ftRight = find_cut(features, cls)
        node = DecisionNode(vid, cut)
        node.left = make_tree((ftLeft, left, minleaf))
        node.right = make_tree((ftRight, right, minleaf))
        
        return node

In [66]:
def score(cls):
    p = sum(cls>0)/float(len(cls))
    return p*(1 - p)
    
def find_cut(features, cls):
    nfeatures = features.shape[1]
    result = []
    for vid in xrange(nfeatures):
        xs = features[:, vid]
        minx, maxx = min(xs), max(xs)
        cuts = np.linspace(minx, maxx, 12)[1: - 1]
        
        for cut in cuts:
            left = cls[xs<cut]
            ftLeft = features[xs<cut]
            right = cls[xs>=cut]
            ftRight = features[xs>=cut]
            s = score(left) + score(right)
            
            result.append((vid, cut, s, left, ftLeft, right, ftRight))
            
    return min(result, key = lambda x: x[2])

In [67]:
new_train.shape

(20548, 4)

In [68]:
ntree = 100
minleaf = 2000

randIdx = np.array([ np.array([np.random.randint(2) for x in xrange(len(new_train))]) for y in xrange(ntree)])
randPop = np.array([ (new_train[idx == 1], classes[idx == 1], minleaf) for idx in randIdx])

In [69]:
start_time = time.time()
p = Pool(4)
tforest = p.map(make_tree, randPop)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1252.26776195 seconds ---


In [70]:
right = 0
wrong = 0

for i, d in enumerate(new_test):
    r = sum([tree.classify(d) for tree in tforest])/len(tforest)
    if r > 0.5:
        if test_classes[i] == 1:
            right += 1
        else:
            wrong += 1
    else:
        if test_classes[i] != 1:
            right += 1
        else:
            wrong += 1

In [71]:
print right, wrong
print right/float((right+wrong))

3088 2061
0.599728102544


In [111]:
def special_pickData(data):
#     useIdx = [0, 1, 5, 6, 7, 13, 15, 16, 17, 18]

    useIdx = [5, 6, 7, 13]
    cIdx = 14
    new_data = []
    classes = []
    noClasses = []
    for i, d in enumerate(data):
        c = d[cIdx]
        new_data.append( [d[idx] for idx in useIdx] )
        classes.append(c)
        
        if c == '':
            noClasses.append(i)
    return np.array(new_data), np.array(classes), np.array(noClasses)

In [112]:
submition_data, submition_classes, noClasses = special_pickData(fdata)

In [113]:
print len(noClasses), len(submition_data)

5000 30697


In [117]:
f = open('submission', 'w')

for i in noClasses:
    r = sum([tree.classify(submition_data[i]) for tree in tforest])/len(tforest)
    f.write("{},{}\n".format(i+1, r))
f.close()

In [101]:
print(len(fdata))

30697
